In [49]:
import os
import pandas as pd
import numpy as np

In [50]:
# macbook
# folder = '/Users/patrickburke/Library/CloudStorage/OneDrive-EmoryUniversity/ECON496RW/processedCSVs/big/pub_pcrevents_chunks/'

# costaPC
folder = r"C:\Users\pat\OneDrive - Emory University\ECON496RW\processedCSVs\big\pub_pcrevents_chunks"

In [51]:
filepaths = [os.path.join(folder, filename) for filename in os.listdir(folder)]
filepaths

['C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk1.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk10.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk2.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk3.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk4.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk5.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk6.csv',
 'C:\\Users\\pat\\OneDrive - Emory University\\ECON496RW\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcreve

In [52]:
def importGender(filepath):
    return pd.read_csv(filepath, usecols=["PcrKey", "ePatient_13"], dtype=np.int32)

In [53]:
df = pd.concat(map(importGender, filepaths))

In [54]:
df

,PcrKey,ePatient_13
0,1472192,7701003
1,13273412,9906003
2,14548133,9906001
3,17184298,9906003
4,21699657,9906003
...,...,...
4999995,168637681,7701001
4999996,168637682,9906003
4999997,168637683,9906003
4999998,168637684,9906001


In [55]:
df = df.rename(columns= {'ePatient_13' : 'Gender'})
# female = 9906001, change to 1
df = df.replace([9906001], 1)
# male = 9906003, change to 0
df = df.replace([9906003], 0)

In [63]:
df.head()

,PcrKey,Gender
0,1472192,7701003
1,13273412,0
2,14548133,1
3,17184298,0
4,21699657,0


In [65]:
# drop missing data rows
df = df[ (df['Gender'] == 0) | (df['Gender'] == 1) ]

In [67]:
df

,PcrKey,Gender
1,13273412,0
2,14548133,1
3,17184298,0
4,21699657,0
5,21719864,1
...,...,...
4999994,168637680,1
4999996,168637682,0
4999997,168637683,0
4999998,168637684,1


In [76]:
# connect to sql db
from sqlalchemy import create_engine
db = create_engine(r"sqlite:///C:\Users\pat\OneDrive - Emory University\ECON496RW\emsData.db")

In [77]:
# create new column
add_col = 'ALTER TABLE socioeconomic ADD gender INTEGER;'
db.execute(add_col)

In [80]:
# we'll just export to new table for now then do the join inside sqlite
#new_table = 'CREATE TABLE genderTable (PcrKey INT PRIMARY KEY, gender INT);'
#db.execute(new_table)

In [82]:
df.to_sql('genderTable', con=db, index=False)

41748496

Join Query:

UPDATE socioeconomic SET gender = (SELECT Gender FROM genderTable WHERE socioeconomic.PcrKey = genderTable.PcrKey);